STEP 1 – Install and Import Libraries

👉 Create a new cell and paste this, then run it:

In [ ]:
import glob
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import numpy as np

print("TF version:", tf.__version__)


Define Image Size and Simple Lane Model

We’ll build a simple MobileNetV2 + upsampling model (MobileNetV3 can be version-dependent, so this is safer and still correct for your project).

In [ ]:
# 1) Define model (fixed upsampling)
# -------------------
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 3

def build_lane_model():
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False  # freeze backbone

    inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

    # Normalization
    x = layers.Rescaling(1./255)(inputs)

    # Backbone (this will reduce 256x256 -> 8x8 feature map)
    x = base_model(x)

    # Simple SCNN-like block
    def spatial_block(t):
        h = layers.Conv2D(64, (1, 3), padding='same', activation='relu')(t)
        v = layers.Conv2D(64, (3, 1), padding='same', activation='relu')(t)
        return layers.Concatenate()([t, h, v])

    x = spatial_block(x)

    # Decoder: UpSample back to 256x256
    # 8x8 -> 16x16
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # 16x16 -> 32x32
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # 32x32 -> 64x64
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # 64x64 -> 128x128
    x = layers.Conv2D(16, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # 128x128 -> 256x256
    x = layers.Conv2D(8, 3, padding='same', activation='relu')(x)
    x = layers.UpSampling2D(size=(2, 2))(x)

    # Final 1-channel mask: 256x256x1
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_lane_model()
model.summary()


Make a Dummy “Fake” Dataset (Just to Test Training)

Right now you don’t yet have lane images loaded, so we’ll create dummy data.
This is just to check that training runs and code is correct.

In [ ]:
# 2) Create dummy data with 256x256 masks
# -------------------
NUM_SAMPLES = 10

X_dummy = np.random.randint(
    0, 256, size=(NUM_SAMPLES, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
).astype('float32')

Y_dummy = np.random.randint(
    0, 2, size=(NUM_SAMPLES, IMG_HEIGHT, IMG_WIDTH, 1)
).astype('float32')

print("X_dummy shape:", X_dummy.shape)
print("Y_dummy shape:", Y_dummy.shape)



Train the Model (Just a Quick Test)

In [ ]:

# 3) Train for 2 epochs
# -------------------
print("\nStarting training...")
history = model.fit(
    X_dummy, Y_dummy,
    epochs=2,
    batch_size=2,
    verbose=1
)

print("\nEpochs actually trained:", len(history.history['loss']))


Test Prediction and Visualize (For Understanding)

In [ ]:
# Take one dummy image
test_img = X_dummy[0:1]  # shape (1, H, W, 3)
pred_mask = model.predict(test_img)[0, ..., 0]  # (H, W)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.title("Input image"); plt.imshow(test_img[0].astype('uint8')); plt.axis("off")
plt.subplot(1,2,2); plt.title("Predicted mask"); plt.imshow(pred_mask, cmap='gray'); plt.axis("off")
plt.show()

making a folder

In [ ]:
!mkdir -p /content/my_lanes/images
!mkdir -p /content/my_lanes/masks


Replace the earlier X_dummy / Y_dummy with this loader:

In [ ]:
import numpy as np

IMG_HEIGHT = 256
IMG_WIDTH = 256

image_paths = sorted(glob.glob("/content/my_lanes/images/*"))
mask_paths  = sorted(glob.glob("/content/my_lanes/masks/*"))

print("Images:", len(image_paths), "Masks:", len(mask_paths))

X_real, Y_real = [], []

for img_p, mask_p in zip(image_paths, mask_paths):
    img = cv2.imread(img_p)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))

    mask = cv2.imread(mask_p, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
    mask = (mask > 128).astype("float32")

    X_real.append(img)
    Y_real.append(mask[..., None])

X_real = np.array(X_real, dtype="float32")
Y_real = np.array(Y_real, dtype="float32")

print("X_real shape:", X_real.shape)
print("Y_real shape:", Y_real.shape)


STEP 8 – Train on Real Data

Now train your existing model (no change needed):

In [ ]:
history = model.fit(
    X_real, Y_real,
    epochs=20,
    batch_size=2
)


STEP 9 – See Real Output

Use the same visualization code as before:

In [ ]:
test_img_batch = X_real[0:100] # Select the first 100 images if it has in input.

for i in range(len(test_img_batch)):
    test_img = test_img_batch[i:i+1]
    pred_mask = model.predict(test_img)[0, ..., 0]

    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1); plt.title(f"Input image {i+1}"); plt.imshow(test_img[0].astype('uint8')); plt.axis("off")
    plt.subplot(1,2,2); plt.title(f"Predicted mask {i+1}"); plt.imshow(pred_mask, cmap='gray'); plt.axis("off")
    plt.show()
